# 神经网络判断最优策略

我们可以利用**深度学习中的神经网络**来自动最优化出最优参数。
方法是这样：
1. 首先将一种颜色定义为1，另一种定义为0
2. 先只考虑第三人的情况，假定前两人都是按照自身的情况来选——这是没有争议的最优解。
3. 我们考虑三个变量，分别是第一个人的选择，第二个人的选择，和自身所抽中的球的颜色，当然都是以0,1表示的。
4. 任意给定三个权重，譬如0.25,0.25,0.5（按照假定，应该是越往后次序抽到的颜色权重越重要，所以这样假设，当然也可以随机假设）。这三者对应前面三个变量的系数，也就是权重。将这三个权重分别与前三个变量乘起来，得到的和，就是对于颜色的预测。所以这三个权重加起来的值为1。预测值大于0.5代表大于1的颜色，小于0.5代表小于1的颜色。
5. 定义一个损失函数，代表预测颜色和实际颜色不一致的错误率。
6. 不断调整权重，使得预测结果尽可能地好。

举例来说，如果第一个选择了红色(1)，第二个人选择了蓝色(0)，你自己抽到了蓝色(0)。初始化的权重是0.25,0.25,0.5。
所以你对于颜色的结果预测是
$$
    \hat{P} = 1 \times 0.25 + 0 \times 0.25 + 0 \times 0.5 = 0.25 < 0 
$$
小于0,即认为是蓝色。
损失函数通常使用交叉熵损失函数。它不仅衡量了猜对的个数，而且要求，颜色为0的预测结果越接近0的，比预测结果在0.4左右的更好。
$$
L(\hat{y},y) = -ylog(\hat{y}) - (1-y)log(1-\hat{y})
$$
调整权重的算法这里使用简单的单层感知器。即找到一个预测错的点，然后权重=权重+$y_i\times x_i$

## 导入库

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fractions
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 仿照上一篇定义类
但是红球定义为1,蓝球定义为0，同时删去Method。

In [ ]:
class Ball:
    def __init__(self, color):
        self.color = color

class Cup:
    def __init__(self, color):
        self.color = color
        if color == 1:
            self.balls = np.array([Ball(1), Ball(1), Ball(0)])
        else:
            self.balls = np.array([Ball(1), Ball(0), Ball(0)])

    def pick(self):
        return self.balls[np.random.randint(0, 3)].color
    
class Cups:
    def __init__(self):
        self.Cups = np.array([Cup(1), Cup(0)])

    def random_pick(self):
        return self.Cups[np.random.randint(0, 2)]
    
class Person:
    def __init__(self, order, Cup):
        self.order = order
        self.choice = None
        self.right = False
        self.cupcolor = Cup.color
        self.picked = Cup.pick()

    def decide(self, info):
        if self.order == 1 or self.order == 2:
            self.choice = self.picked
        else
            self.choice = None
        
    def strategy(self, info, ):
        self.decide(info, )
        if self.choice == self.cupcolor:
            self.right = True

class Test:
    def __init__(self, Cups):
        self.cup = Cups.random_pick()
        People = []
        for i in range(1, 7):
            People.append(Person(i, self.cup))
        self.People = np.array(People)
        self.info = []
        
    def test(self,):
        for i in range(0, 6):
            self.People[i].strategy(self.info,)
            self.info.append(self.People[i].choice)

    def test_result(self,rounds,):
        self.test()
        data = pd.DataFrame([[rounds, 1, self.People[0].picked, self.People[0].choice, self.People[0].right]])
        for i in range(1, 6):
            choice = pd.DataFrame([[rounds, i + 1, self.People[i].picked, self.People[i].choice, self.People[i].right]])
            data = data.append(choice)
        return data
    
class Multi_test:
    def __init__(self, num, ):
        self.cups = Cups()
        self.data = pd.DataFrame(Test(self.cups).test_result(1,))
        for i in range(1, num):
            self.data = self.data.append(Test(self.cups).test_result(i+1,))
        self.data.columns=['round', 'order', 'picked', 'choice', 'right']

    def result(self):
        return self.data